Generating Data
===============

We first create 200 random two-dimensional data points.
The data points are sampled from a multinomial normal distribution.

You don't have to understand precisely how we do this. 

In [55]:
import numpy as np
import matplotlib.pyplot as plt
Cov = np.array([[2.9, -2.2], [-2.2, 6.5]])
X = np.random.multivariate_normal([1,2], Cov, size=200)

Let's have a look at the raw data first...

In [56]:
np.set_printoptions(4, suppress=True) # show only four decimals
print(X[:10,:]) # print the first 10 rows of X (from 0 to 9)

[[ 2.5954  3.687 ]
 [ 0.164   4.2647]
 [ 3.789   3.4515]
 [ 0.198   2.9585]
 [-0.0593  4.6058]
 [ 4.3778  2.0349]
 [-0.1215  2.2444]
 [-1.0884  7.9158]
 [ 0.7549  2.7213]
 [ 1.4439  0.8954]]


Do you see a relationship between the two columns? Tricky, I'd say. However, since the data is two-dimensional, we can plot the data, which allows us to see the relationship.

In [57]:
plt.figure(figsize=(4,4))
plt.scatter(X[:,0], X[:,1])
plt.axis('equal') # equal scaling on both axis;
plt.show()

We can also have a look at the actual covariance matrix:

In [58]:
print(np.cov(X,rowvar=False))

[[ 2.893  -2.0963]
 [-2.0963  6.3124]]


Running PCA
===========

We would now like to analyze the directions in which the data varies most. For that, we 

1. place the point cloud in the center (0,0) and
2. rotate it, such that the direction with most variance is parallel to the x-axis.

Both steps can be done using PCA, which is conveniently available in sklearn.

We start by loading the PCA class from the sklearn package and creating an instance of the class:

In [59]:
from sklearn.decomposition import PCA
pca = PCA()

Now, `pca` is an object which has a function `pca.fit_transform(x)` which performs both steps from above to its argument `x`, and returns the centered and rotated version of `x`.

In [60]:
X_pca = pca.fit_transform(X)

In [61]:
pca.components_

array([[ 0.4289, -0.9033],
       [ 0.9033,  0.4289]])

In [62]:
pca.mean_

array([ 0.9075,  1.8961])

In [63]:
plt.figure(figsize=(4,4))
plt.scatter(X_pca[:,0], X_pca[:,1])
plt.axis('equal');
plt.show()

The covariances between different axes should be zero now. We can double-check by having a look at the non-diagonal entries of the covariance matrix:

In [64]:
print(np.cov(X_pca, rowvar=False))

[[ 7.3078 -0.    ]
 [-0.      1.8977]]


High-Dimensional Data
=====================

Our small example above was very easy, since we could get insight into the data by simply plotting it. This approach will not work once you have more than 3 dimensions, let's say we have the same data, but it is represented in four dimensions:

In [65]:
np.random.seed(1)
X_HD = np.dot(X,np.random.uniform(0.2,3,(2,4))*(np.random.randint(0,2,(2,4))*2-1))

Lets look at the data again. First, the raw data:

In [66]:
print(X_HD[:10,:])

[[ -5.8021   7.4444  -3.1802  -7.021 ]
 [ -2.8297   2.3192  -3.11    -5.151 ]
 [ -7.2907   9.9825  -3.2494  -7.9952]
 [ -2.0782   1.7955  -2.1743  -3.6615]
 [ -2.7327   1.9805  -3.3114  -5.3156]
 [ -7.2304  10.6382  -2.3452  -6.9573]
 [ -1.2049   0.7597  -1.595   -2.4933]
 [ -3.3474   1.217   -5.4935  -8.1033]
 [ -2.695    2.9215  -2.1147  -3.9674]
 [ -2.5217   3.6115  -0.9353  -2.5564]]


That one is more tricky. See anything? We can also try plot a few two-dimensional projections:

In [67]:
# 
plt.figure(figsize=(8,8))
for i in range(4):
    for j in range(4):
        plt.subplot(4, 4, i * 4 + j + 1)
        plt.scatter(X_HD[:,i], X_HD[:,j])
        plt.axis('equal')
        plt.gca().set_aspect('equal')
plt.show()

It is not easy to see that this is still a two-dimensional dataset! 

However, if we now do PCA on it, you'll see that the last two dimensions do not matter at all:

In [68]:
X_HE = pca.fit_transform(X_HD)
print(X_HE[:10,:])

[[ -6.9949   1.0039   0.       0.    ]
 [ -0.9513   2.4454   0.      -0.    ]
 [-10.0233   0.3401  -0.       0.    ]
 [  0.6223   1.2382   0.      -0.    ]
 [ -0.764    2.8413   0.      -0.    ]
 [ -9.8563  -1.1765   0.      -0.    ]
 [  2.4254   0.7051  -0.      -0.    ]
 [ -2.1209   6.2549  -0.       0.    ]
 [ -0.6283   0.812   -0.      -0.    ]
 [ -0.2179  -1.1166  -0.       0.    ]]


Here it is easy to see, that the data is **still only two-dimensional**. Let's plot the two dimensions.

In [69]:
plt.figure(figsize=(4,4))
plt.scatter(X_HE[:,0], X_HE[:,1])
plt.axis('equal')
plt.gca().set_aspect('equal')
plt.show()

Why does the result look differently than the two-dimensional data from which we generated it?

In [70]:
plt.figure(figsize=(8,8))
for i in range(4):
    for j in range(4):
        plt.subplot(4, 4, i * 4 + j + 1)
        plt.scatter(X_HE[:,i], X_HE[:,j])
        plt.gca().set_xlim(-40,40)
        plt.gca().set_ylim(-40,40)
        plt.axis('equal')
        plt.gca().set_aspect('equal')
plt.show()

Dimension Reduction with PCA
============================

We can see that there are actually only two dimensions in the dataset. 

Let's throw away even more data -- the second dimension -- and reconstruct the original data in `D`.

In [71]:
pca = PCA(1) # only keep one dimension!
X_E = pca.fit_transform(X_HD)
print(X_E[:10,:])

[[ -6.9949]
 [ -0.9513]
 [-10.0233]
 [  0.6223]
 [ -0.764 ]
 [ -9.8563]
 [  2.4254]
 [ -2.1209]
 [ -0.6283]
 [ -0.2179]]


Now lets plot the reconstructed data and compare to the original data D. We plot the original data in red, and the reconstruction with only one dimension in blue:

In [72]:
X_reconstructed = pca.inverse_transform(X_E)
plt.figure(figsize=(8,8))
for i in range(4):
    for j in range(4):
        plt.subplot(4, 4, i * 4 + j + 1)
        plt.scatter(X_HD[:,i], X_HD[:,j],c='r')
        plt.scatter(X_reconstructed[:,i], X_reconstructed[:,j],c='b')
        plt.axis('equal')
plt.show()

## PCA on Images

In this final example, we use the $k$-Means algorithm on the classical MNIST dataset.

The MNIST dataset contains images of hand-written digits. 

Let's first fetch the dataset from the internet (which may take a while, note the asterisk [*]):

In [73]:
from sklearn.datasets import fetch_mldata
from sklearn.cluster import KMeans
from sklearn.utils import shuffle
#X_digits, _,_, Y_digits = fetch_mldata("MNIST Original").values() # fetch dataset from internet
#X_digits, Y_digits = shuffle(X_digits,Y_digits) # shuffle dataset (which is ordered!)
#X_digits = X_digits[-5000:]       # take only the last instances, to shorten runtime of PCA
# 26.5.16 MH needed to change to this:
mnist = fetch_mldata('MNIST original')
mnist.data, mnist.target = shuffle(mnist.data,mnist.target)
X_digits=mnist.data[-5000:]


Let's have a look at some of the instances in the dataset we just loaded:

In [74]:
plt.rc("image", cmap="binary")
plt.figure(figsize=(8,4))
for i in range(10):
    plt.subplot(2,5,i+1)
    plt.imshow(X_digits[i].reshape(28,28))
    plt.xticks(())
    plt.yticks(())
plt.tight_layout()
plt.show()

**Warning**: This takes quite a few seconds, so be patient until the asterisk [*] disappears!

In [75]:
from sklearn.decomposition import PCA
pca = PCA()
X2_digits = pca.fit_transform(X_digits)

It does not make much sense to look at the transformed images, they will look like noise to us. 

Instead, let's have a look at the most important directions on which the dataset was projected:

In [76]:
plt.figure(figsize=(8,6))
W = pca.components_

for i in range(10): # loop over all means
    plt.subplot(2,5,i+1)
    plt.imshow(W[i].reshape(28,28))
    plt.xticks(())
    plt.yticks(())
plt.tight_layout()
plt.show()

The later directions (here, from the 100-th on) mainly show noise, small variations between different, but very similar training instances:

In [77]:
plt.figure(figsize=(8,4))
W = pca.components_

for i in range(10): # loop over all means
    plt.subplot(2,5,i+1)
    plt.imshow(W[100+i].reshape(28,28))
    plt.xticks(())
    plt.yticks(())
plt.tight_layout()
plt.show()

The number of "interesting" dimensions can be seen from the importance of the found directions. 

We can simply plot them:

In [78]:
plt.plot(pca.explained_variance_);
plt.show()
print(len(pca.explained_variance_))

784


We can see that the intrinsic dimensionality is not higher than maybe 100, even though the dataset has 784 dimensions!

Let's reconstruct the data again using only a handfull of the 784 dimensions:

In [79]:
from sklearn.decomposition import PCA
pca = PCA(20)
X2_few_digits = pca.fit_transform(X_digits)

In [80]:
plt.figure(figsize=(16,4))
X_recons_digits = pca.inverse_transform(X2_few_digits)
for i in range(10):
    plt.subplot(2,10,i+1)
    plt.imshow(X_recons_digits[i].reshape(28,28))
    plt.xticks(())
    plt.yticks(())
for i in range(10):
    plt.subplot(2,10,11+i)
    plt.imshow(X_digits[i].reshape(28,28))
    plt.xticks(())
    plt.yticks(())
plt.tight_layout()
plt.show()

# Playing around with this Notebook

- What happens, when you multiply one of the data axis with a large (or small) number? 

  e.g. using X[:,0] *= 100

  Does the result stay the same? Why/why not?

-----

- Try to explore the iris dataset below using PCA. 

  What happens if you visualize the *last* components of PCA instead of the first ones?

-----

- Use PCA with 1, 2 or 3 axes and reconstruct the Iris data from each. How do the results change? Show plots!

In [81]:
from sklearn import datasets
_,data,target,_,_ = datasets.load_iris().values()